In [1]:
import torch
assert torch.__version__>='1.2.0', 'Expect PyTorch>=1.2.0 but get {}'.format(torch.__version__)
from torch import nn
import torch.nn.functional as F

import numpy as np

import time
import os
import sys
import pickle

imp_dir = '../Implementations'
sys.path.insert(1, imp_dir)
data_dir = '../Data/criteo'
sys.path.insert(1, data_dir)

import logging
import importlib
importlib.reload(logging)

logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(levelname)-6s %(message)s', level=logging.INFO, datefmt='%H:%M:%S')
logger = logging.getLogger(__name__)

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info('Device in Use: {}'.format(DEVICE))

05:25:58 INFO   Device in Use: cuda


## Prepare Data

### List all available files

In [3]:
np_artifact_dir = os.path.join(data_dir, 'criteo_train_numpy_artifact')
index_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='index', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
value_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='value', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
label_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='label', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))

In [ ]:
start = time.time()

train_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[:10]]),
)

logger.info('Training data loaded after {:.2f}s'.format(time.time()-start))

start = time.time()

test_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[10:]]),
)

logger.info('Test data loaded after {:.2f}s'.format(time.time()-start))

## Create Model

In [ ]:
import execution
importlib.reload(execution)
import DeepFM_BinClf_Torch 
importlib.reload(DeepFM_BinClf_Torch)

In [ ]:
embedding_map_dict_pkl_path = os.path.join(data_dir, 'criteo_feature_dict_artifact/categorical_feature_map_dict.pkl')
with open(embedding_map_dict_pkl_path, 'rb') as f:
    embedding_map_dict = pickle.load(f)

In [ ]:
DeepFM = DeepFM_BinClf_Torch.DeepFM_2D_Layer(len(embedding_map_dict)+60,
                                             39,
                                             10, 
                                             [400 for _ in range(3)], 
                                             [0, 0], 
                                             [0.5 for _ in range(4)]).to(DEVICE)

## Sample Run

In [ ]:
cwd = os.getcwd()
checkpoint_dir = os.path.join(cwd, 'DeepFM_artifact')
checkpoint_prefix = 'DeepFM'

In [ ]:
execution.train_model(DeepFM, 
                      train_data, 
                      test_data, 
                      F.binary_cross_entropy_with_logits, 
                      torch.optim.Adam(DeepFM.parameters()), 
                      DEVICE, 
                      checkpoint_dir, 
                      checkpoint_prefix)

## Model Performance

In [30]:
os.listdir(checkpoint_dir)

['DeepFM_1', 'DeepFM_2']

In [33]:
model = DeepFM_BinClf_Torch.DeepFM_2D_Layer(len(embedding_map_dict)+60,
                                             39,
                                             10, 
                                             [400 for _ in range(3)], 
                                             [0, 0], 
                                             [0.5 for _ in range(4)]).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DeepFM_2')))
model.eval()
model = model.to(DEVICE)

In [34]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score(model, train_data, DEVICE)))

05:25:41 INFO   Model ROC Score: 0.826445


In [15]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score(model, test_data, DEVICE)))

00:49:53 INFO   Model ROC Score: 0.792765
